
# BIN PACKING PROBLEM
<font color='orange'>The Bin Packing Proble (BPP) component entails packing the $n$ deliverable items into the minimum number of bins without exceeding its fixed capacity and has the minimum wastage of space.
 In this case our bins are the subset of available delivery vehicles, into which the items are to be packed.
Note, not all the items that are required to be distributed are the same in size.</font>

# One dimesional
<font color='grass'>
In order to simplify the problem, we initially consider the problem to be one dimensional. Only one dimension of the item is not fixed, for example length, whilst the other two dimensions (width and height) remain constant.
</font>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math 
import csv as csv

### Read-in and check data

In [2]:
items = 'items'
bins = 'bins'

def read_data(fileName):
    df = pd.read_csv(fileName)
    return df
    
def check_packaging(df):
    rows, cols = df.shape #size of the data set
    return (rows, cols)

def data_check(df, n=3):
    df_top_n = df.head(n)
    return (df_top_n)

def check_ns(df):
    ns = df.describe()
    return ns


### ITEM DATA:

In [3]:
df_items = read_data('%s.csv'%items)
print("rows(%s) x cols(%s) "%check_packaging(df_items))
print()
print("%s"%data_check(df_items))
print()
print(check_ns(df_items))
print()
df_items.set_index('item')

rows(7) x cols(3) 

   item  size  bin
0     0     4  NaN
1     1     3  NaN
2     2     6  NaN

           item      size  bin
count  7.000000  7.000000  0.0
mean   3.000000  3.000000  NaN
std    2.160247  1.632993  NaN
min    0.000000  1.000000  NaN
25%    1.500000  2.000000  NaN
50%    3.000000  3.000000  NaN
75%    4.500000  3.500000  NaN
max    6.000000  6.000000  NaN



,size,bin
item,,
0,4,NaN
1,3,NaN
2,6,NaN
3,3,NaN
4,2,NaN
5,1,NaN
6,2,NaN


### Vehicle or Bin data:

In [4]:
df_vehicles = read_data('%s.csv'%bins)
print("VEHICLE DETAILS:")
print("rows(%s) x cols(%s) "%check_packaging(df_vehicles))
print()
print("%s"%data_check(df_vehicles))
print()
print(check_ns(df_vehicles))
print()
df_vehicles.set_index('bin')

VEHICLE DETAILS:
rows(3) x cols(3) 

   bin  size  available_space
0    0     9                9
1    1     3                3
2    2     4                4

       bin      size  available_space
count  3.0  3.000000         3.000000
mean   1.0  5.333333         5.333333
std    1.0  3.214550         3.214550
min    0.0  3.000000         3.000000
25%    0.5  3.500000         3.500000
50%    1.0  4.000000         4.000000
75%    1.5  6.500000         6.500000
max    2.0  9.000000         9.000000



,size,available_space
bin,,
0,9,9
1,3,3
2,4,4


# GREEDY APPROACH
<font color='orange'>A technique which makes locally optimal choices with hope of obtaining the global optimum. 
</font>

## Fit algorithms:  

### First-fit algorithm:
<font color='orange'>
Algorithm scans the bins for the first bin which has a large enough space to fit the item. If the current bin has adequate space, the items is allocated to the bin, else the next bin is checked. For the next item, we iterate the bins from the FIRST bin (hence first fit algorithm).
</font>

In [5]:
def first_fit(df_items, df_vehicles):
    number_of_items = df_items.shape[0]
    number_of_bins = df_vehicles.shape[0]
    for i in range(number_of_items):
        j = 0
        item_allocated = False
        item_size = df_items.iat[i,1]
        while j<= number_of_bins and item_allocated == False:
            available_bin_space = df_vehicles.iat[j,2]
            if available_bin_space >= item_size: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.iat[j,2] = df_vehicles.iat[j,2] - item_size #update avialable space
                bin_num = df_vehicles.iat[j,0]
                df_items.iat[i,2] = bin_num#set the allocated bin for the item
            elif available_bin_space < item_size: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    item_allocated = True
                    df_items.iat[i,2] = np.nan
    return df_items, df_vehicles

In [6]:
#bin summary
def bin_summary(df_vehicles):
    number_of_bins = df_vehicles.shape[0]
    unused = 0
    partial = 0
    max_fill = 0
    for i in range(number_of_bins):
        av_space = df_vehicles.iloc[i][2]
        bin_size = df_vehicles.iloc[i][1]
        if av_space == bin_size:
            unused = unused + 1
        elif av_space == 0:
            max_fill = max_fill +1
        elif av_space < bin_size and av_space!=0:
            partial = partial+1
            
    print("Number of bins : %d" %(number_of_bins))
    print("Number of partial filled bins : %d" %(partial))
    print("Number of unused bins : %d" %(unused))
    print("Number of max filled bins : %d" %(max_fill))
    return None

#### First-fit algorithm: UNSORTED LISTS

In [7]:
df_items_copy1 = df_items.copy()
df_items_copy1

,item,size,bin
0,0,4,NaN
1,1,3,NaN
2,2,6,NaN
3,3,3,NaN
4,4,2,NaN
5,5,1,NaN
6,6,2,NaN


In [8]:
df_vehicles_copy1 = df_vehicles.copy()
df_vehicles_copy1

,bin,size,available_space
0,0,9,9
1,1,3,3
2,2,4,4


In [9]:
df_items_copy1, df_vehicles_copy1 = first_fit(df_items_copy1, df_vehicles_copy1)

In [10]:
print('Item allocation to bins:')
df_items_copy1

Item allocation to bins:


,item,size,bin
0,0,4,0.0
1,1,3,0.0
2,2,6,NaN
3,3,3,1.0
4,4,2,0.0
5,5,1,2.0
6,6,2,2.0


In [11]:
print('Remaining space in bins:')
df_vehicles_copy1

Remaining space in bins:


,bin,size,available_space
0,0,9,0
1,1,3,0
2,2,4,1


In [12]:
bin_summary(df_vehicles_copy1)

Number of bins : 3
Number of partial filled bins : 1
Number of unused bins : 0
Number of max filled bins : 2


#### First-fit algorithm:  DESCENDING SORTED LISTS of bins and items.

In [13]:
df_items_copy2 = df_items.copy()
df_items_copy2 = df_items_copy2.sort_values(by='size', ascending=0)
df_items_copy2 = df_items_copy2.reset_index(drop=True)#NOTE: We reset the index otherwise index items as original set, the item number and index value are now different!
df_items_copy2

,item,size,bin
0,2,6,NaN
1,0,4,NaN
2,1,3,NaN
3,3,3,NaN
4,4,2,NaN
5,6,2,NaN
6,5,1,NaN


In [14]:
df_vehicles_copy2 = df_vehicles.copy()
df_vehicles_copy2 = df_vehicles_copy2.sort_values(by='size', ascending=0)
df_vehicles_copy2 = df_vehicles_copy2.reset_index(drop=True)
df_vehicles_copy2.set_index('bin')
df_vehicles_copy2

,bin,size,available_space
0,0,9,9
1,2,4,4
2,1,3,3


In [15]:
df_items_copy2, df_vehicles_copy2 = first_fit(df_items_copy2, df_vehicles_copy2)

In [16]:
df_items_copy2

,item,size,bin
0,2,6,0.0
1,0,4,2.0
2,1,3,0.0
3,3,3,1.0
4,4,2,NaN
5,6,2,NaN
6,5,1,NaN


In [17]:
df_vehicles_copy2

,bin,size,available_space
0,0,9,0
1,2,4,0
2,1,3,0


In [18]:
bin_summary(df_vehicles_copy2)

Number of bins : 3
Number of partial filled bins : 0
Number of unused bins : 0
Number of max filled bins : 3


#### First-fit algorithm: ASCENDING  SORTED item and bin list.

In [19]:
df_items_copy3 = df_items.copy()
df_items_copy3 = df_items_copy3.sort_values(by='size', ascending=1)
df_items_copy3 = df_items_copy3.reset_index(drop=True)
df_items_copy3

,item,size,bin
0,5,1,NaN
1,4,2,NaN
2,6,2,NaN
3,1,3,NaN
4,3,3,NaN
5,0,4,NaN
6,2,6,NaN


In [20]:
df_vehicles_copy3 = df_vehicles.copy()
df_vehicles_copy3 = df_vehicles_copy3.sort_values(by='size', ascending=1).reset_index(drop=True)
df_vehicles_copy3

,bin,size,available_space
0,1,3,3
1,2,4,4
2,0,9,9


In [21]:
df_items_copy3, df_vehicles_copy3 = first_fit(df_items_copy3, df_vehicles_copy3)


In [22]:
df_items_copy3

,item,size,bin
0,5,1,1.0
1,4,2,1.0
2,6,2,2.0
3,1,3,0.0
4,3,3,0.0
5,0,4,NaN
6,2,6,NaN


In [23]:
df_vehicles_copy3

,bin,size,available_space
0,1,3,0
1,2,4,2
2,0,9,3


In [24]:
bin_summary(df_vehicles_copy3)

Number of bins : 3
Number of partial filled bins : 2
Number of unused bins : 0
Number of max filled bins : 1


### Next-fit algorithm
<font color='orange'>After allocating first bin large enough to house the item, when looking at the next item to be allocated, find the next suitable bin from the current bin. NOT starting from the very first bin. Check from the next bin in a loop, and stop at the bin before the last allocated (current bin). The search space is from current bin to the previous in a loop.

In [25]:
def next_fit(df_items, df_vehicles):
    number_of_items = df_items.shape[0]
    number_of_bins = df_vehicles.shape[0]
    j = 0
    for i in range(number_of_items):
        item_allocated = False
        item_size = df_items.iat[i,1]
        while item_allocated == False and j<= number_of_bins:
            available_bin_space = df_vehicles.iat[j,2]
            if available_bin_space >= item_size: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.iat[j,2] = df_vehicles.iat[j,2] - item_size #update avialable space
                bin_num = df_vehicles.iat[j,0]
                df_items.iat[i,2] = bin_num #set the allocated bin for the item
                j_curr = j
            elif available_bin_space < item_size: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    j = 0
                    if j_curr == j:
                        item_allocated = True
                        df_items.iat[i,2] = np.nan
                        j = j_curr
    return df_items, df_vehicles

In [26]:
df_items_copy4 = df_items.copy()
df_items_copy4

,item,size,bin
0,0,4,NaN
1,1,3,NaN
2,2,6,NaN
3,3,3,NaN
4,4,2,NaN
5,5,1,NaN
6,6,2,NaN


In [27]:
df_vehicles_copy4 = df_vehicles.copy()
df_vehicles_copy4

,bin,size,available_space
0,0,9,9
1,1,3,3
2,2,4,4


In [28]:
df_items_copy4, df_vehicles_copy4 = next_fit(df_items_copy4, df_vehicles_copy4)

In [29]:
df_items_copy4

,item,size,bin
0,0,4,0.0
1,1,3,0.0
2,2,6,NaN
3,3,3,1.0
4,4,2,2.0
5,5,1,2.0
6,6,2,0.0


In [30]:
df_vehicles_copy4

,bin,size,available_space
0,0,9,0
1,1,3,0
2,2,4,1


In [31]:
bin_summary(df_vehicles_copy4)

Number of bins : 3
Number of partial filled bins : 1
Number of unused bins : 0
Number of max filled bins : 2


### Best-fit algorithm
<font color='orange'>Allocating item to a in such that there is minimum wastage of space is left in in the bin.
<\font>

In [32]:
def best_fit(df_items, df_vehicles):
    number_of_items = df_items.shape[0]
    number_of_bins = df_vehicles.shape[0]
    for i in range(number_of_items):
        j = 0
        item_allocated = False
        item_size = df_items.iat[i,1]
        df_vehicles = df_vehicles.sort_values(by='available_space', ascending=1).reset_index(drop=True)#after each iteration, order bins as per min space, so less wastage
        while j<= number_of_bins and item_allocated == False:
            available_bin_space = df_vehicles.iat[j,2]
            if available_bin_space >= item_size: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.iat[j,2] = df_vehicles.iat[j,2] - item_size #update avialable space
                bin_num = df_vehicles.iat[j,0]
                df_items.iat[i,2] = bin_num#set the allocated bin for the item
            elif available_bin_space < item_size: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    item_allocated = True
                    df_items.iat[i,2] = np.nan
    return df_items, df_vehicles

In [33]:
df_items_copy5 = df_items.copy()
df_items_copy5

,item,size,bin
0,0,4,NaN
1,1,3,NaN
2,2,6,NaN
3,3,3,NaN
4,4,2,NaN
5,5,1,NaN
6,6,2,NaN


In [34]:
df_vehicles_copy5 = df_vehicles.copy()
df_vehicles_copy5

,bin,size,available_space
0,0,9,9
1,1,3,3
2,2,4,4


In [35]:
df_items_copy5, df_vehicles_copy5 = best_fit(df_items_copy5, df_vehicles_copy5)

In [36]:
df_items_copy5

,item,size,bin
0,0,4,2.0
1,1,3,1.0
2,2,6,0.0
3,3,3,0.0
4,4,2,NaN
5,5,1,NaN
6,6,2,NaN


In [37]:
df_vehicles_copy5

,bin,size,available_space
0,2,4,0
1,1,3,0
2,0,9,0


In [38]:
bin_summary(df_vehicles_copy5)

Number of bins : 3
Number of partial filled bins : 0
Number of unused bins : 0
Number of max filled bins : 3


### Worst-fit algorithm
<font color='orange'>Allocating item to a bin such that there is maximum wastage of space is left in in the bin.
<\font>

In [39]:
def worst_fit(df_items, df_vehicles):
    number_of_items = df_items.shape[0]
    number_of_bins = df_vehicles.shape[0]
    for i in range(number_of_items):
        j = 0
        item_allocated = False
        item_size = df_items.iat[i,1]
        df_vehicles = df_vehicles.sort_values(by='available_space', ascending=0).reset_index(drop=True)#after each iteration, order bins as per min space, so less wastage
        while j<= number_of_bins and item_allocated == False:
            available_bin_space = df_vehicles.iat[j,2]
            if available_bin_space >= item_size: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.iat[j,2] = df_vehicles.iat[j,2] - item_size #update avialable space
                bin_num = df_vehicles.iat[j,0]
                df_items.iat[i,2] = bin_num#set the allocated bin for the item
            elif available_bin_space < item_size: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    item_allocated = True
                    df_items.iat[i,2] = np.nan
    return df_items, df_vehicles

In [40]:
df_items_copy6 = df_items.copy()
df_items_copy6

,item,size,bin
0,0,4,NaN
1,1,3,NaN
2,2,6,NaN
3,3,3,NaN
4,4,2,NaN
5,5,1,NaN
6,6,2,NaN


In [41]:
df_vehicles_copy6 = df_vehicles.copy()
df_vehicles_copy6

,bin,size,available_space
0,0,9,9
1,1,3,3
2,2,4,4


In [42]:
df_items_copy6, df_vehicles_copy6 = worst_fit(df_items_copy6, df_vehicles_copy6)

In [43]:
df_items_copy6

,item,size,bin
0,0,4,0.0
1,1,3,0.0
2,2,6,NaN
3,3,3,2.0
4,4,2,1.0
5,5,1,0.0
6,6,2,NaN


In [44]:
df_vehicles_copy6

,bin,size,available_space
0,0,9,1
1,1,3,1
2,2,4,1


In [45]:
bin_summary(df_vehicles_copy6)

Number of bins : 3
Number of partial filled bins : 3
Number of unused bins : 0
Number of max filled bins : 0


In [46]:
#to shuffle dataframe
from sklearn.utils import shuffle 
df_items_copy7 = shuffle(df_items.copy())
df_items_copy7 = df_items_copy7.reset_index(drop=True)#NOTE: We reset the index otherwise index items as original set, the item number and index value are now different!
df_items_copy7


,item,size,bin
0,2,6,NaN
1,4,2,NaN
2,6,2,NaN
3,3,3,NaN
4,1,3,NaN
5,5,1,NaN
6,0,4,NaN


In [47]:
df_vehicles_copy7 = df_vehicles.copy()
df_vehicles_copy7

,bin,size,available_space
0,0,9,9
1,1,3,3
2,2,4,4


In [48]:
df_items_copy7, df_vehicles_copy7 = first_fit(df_items_copy7,df_vehicles_copy7)

In [49]:
df_items_copy7

,item,size,bin
0,2,6,0.0
1,4,2,0.0
2,6,2,1.0
3,3,3,2.0
4,1,3,NaN
5,5,1,0.0
6,0,4,NaN


In [50]:
df_vehicles_copy7

,bin,size,available_space
0,0,9,0
1,1,3,1
2,2,4,1


In [51]:
df_items_copy7 = df_items_copy7.sort_values(by = 'item').reset_index(drop=True)
df_items_copy7


,item,size,bin
0,0,4,NaN
1,1,3,NaN
2,2,6,0.0
3,3,3,2.0
4,4,2,0.0
5,5,1,0.0
6,6,2,1.0


In [79]:
newDF = pd.DataFrame()

In [80]:
newDF = newDF.append(df_items_copy7[:][['item','size']], ignore_index=False)

In [81]:
newDF

,item,size
0,0,4
1,1,3
2,2,6
3,3,3
4,4,2
5,5,1
6,6,2


In [82]:
newDF.append(df_items_copy7[:][['bin']])

,bin,item,size
0,NaN,0.0,4.0
1,NaN,1.0,3.0
2,NaN,2.0,6.0
3,NaN,3.0,3.0
4,NaN,4.0,2.0
5,NaN,5.0,1.0
6,NaN,6.0,2.0
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,0.0,NaN,NaN
